# Frustum PointNets PyTorch Implementation

In [2]:
import torch
import torchvision
import torch.nn as nn


import matplotlib.pyplot

In [ ]:
class DetectionNet:
    def __init__(in_planes, out_planes, )